### Imports

In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import parallel_coordinates

In [2]:
# setting up pandas options
pd.set_option('display.max_colwidth', None)

### Loading data

In [3]:
path = '~/ecomm-open-cdp/2019-Oct.csv'
header = pd.read_csv(path, index_col=0, nrows=1)
header

,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
event_time,,,,,,,,
2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c


In [4]:
# filtering on event_type = purchase and category_code not na
data_chunks = pd.read_csv(path, iterator = True, chunksize = 10000)
store_data = pd.concat([ chunk[(~chunk.category_code.isna()) & (chunk.event_type == 'purchase')] for chunk in data_chunks])

In [5]:
store_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 569424 entries, 162 to 42448657
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     569424 non-null  object 
 1   event_type     569424 non-null  object 
 2   product_id     569424 non-null  int64  
 3   category_id    569424 non-null  int64  
 4   category_code  569424 non-null  object 
 5   brand          549507 non-null  object 
 6   price          569424 non-null  float64
 7   user_id        569424 non-null  int64  
 8   user_session   569424 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 43.4+ MB


In [6]:
store_data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
162,2019-10-01 00:02:14 UTC,purchase,1004856,2053013555631882655,electronics.smartphone,samsung,130.76,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564
308,2019-10-01 00:04:37 UTC,purchase,1002532,2053013555631882655,electronics.smartphone,apple,642.69,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68
442,2019-10-01 00:07:07 UTC,purchase,13800054,2053013557418656265,furniture.bathroom.toilet,santeri,54.42,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f
574,2019-10-01 00:09:26 UTC,purchase,4804055,2053013554658804075,electronics.audio.headphone,apple,189.91,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553
603,2019-10-01 00:09:54 UTC,purchase,4804056,2053013554658804075,electronics.audio.headphone,apple,161.98,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68


In [7]:
store_data.shape

(569424, 9)

In [8]:
#saving filtered data
store_data.to_csv('~/ecomm-open-cdp/2019-csv-filt.csv')

### Filtetring data

In [9]:
FILT_COLS = ['user_id', 'user_session', 'category_code']

In [10]:
filt_data = store_data[FILT_COLS]
filt_data.head()

,user_id,user_session,category_code
162,543272936,8187d148-3c41-46d4-b0c0-9c08cd9dc564,electronics.smartphone
308,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,electronics.smartphone
442,555332717,1dea3ee2-2ded-42e8-8e7a-4e2ad6ae942f,furniture.bathroom.toilet
574,524601178,2af9b570-0942-4dcd-8f25-4d84fba82553,electronics.audio.headphone
603,551377651,3c80f0d6-e9ec-4181-8c5c-837a30be2d68,electronics.audio.headphone


In [11]:
trans_groups = filt_data.groupby(['user_id','user_session'])

In [12]:
print('Total transactions in the dataset', trans_groups.ngroups)

Total transactions in the dataset 483608


In [13]:
group_sizes =  trans_groups.size()
group_sizes

user_id    user_session                        
303160429  8957377b-66b3-4661-ae91-5c2b5e29fd0e    1
340041246  596ea40e-80c7-4c30-bca4-60513fec1925    1
384989212  7472f245-e885-4c5e-b3b1-476a7b19b508    1
401021311  a47a58c4-808a-480f-b116-7e15cdeffd05    1
403013066  10456526-1e4c-487b-824b-04dd0d1b73d1    2
                                                  ..
566265448  926bbe7d-e295-4c6b-a635-c9ea804b8f84    1
566267483  875bbd0f-88c9-4ff1-afd3-b5d7ba4fd32a    2
566270060  3c542b6f-f1f5-42cb-8c3e-809f9d4e86bc    2
566274637  1055400a-55b5-4d1e-9152-92e25c7e9266    1
566278294  88aa1f8b-8a36-4767-a0e2-2b984c2190c1    1
Length: 483608, dtype: int64

In [14]:
#Transaction aggregator
def transaction_agg(df):
    items = list(set(df.category_code))
    return pd.DataFrame({'items':[items]})

In [15]:
#aggregating transactions across userids and sessionids
transactions_df = trans_groups.apply(transaction_agg).reset_index()

In [16]:
transactions_df

,user_id,user_session,level_2,items
0,303160429,8957377b-66b3-4661-ae91-5c2b5e29fd0e,0,[electronics.clocks]
1,340041246,596ea40e-80c7-4c30-bca4-60513fec1925,0,[electronics.video.tv]
2,384989212,7472f245-e885-4c5e-b3b1-476a7b19b508,0,[appliances.kitchen.oven]
3,401021311,a47a58c4-808a-480f-b116-7e15cdeffd05,0,[electronics.video.tv]
4,403013066,10456526-1e4c-487b-824b-04dd0d1b73d1,0,[electronics.smartphone]
...,...,...,...,...
483603,566265448,926bbe7d-e295-4c6b-a635-c9ea804b8f84,0,[electronics.clocks]
483604,566267483,875bbd0f-88c9-4ff1-afd3-b5d7ba4fd32a,0,"[kids.toys, computers.peripherals.mouse]"
483605,566270060,3c542b6f-f1f5-42cb-8c3e-809f9d4e86bc,0,"[appliances.kitchen.microwave, electronics.smartphone]"
483606,566274637,1055400a-55b5-4d1e-9152-92e25c7e9266,0,[electronics.smartphone]


In [17]:
transactions = list(transactions_df['items'])

In [18]:
#one hot encoding transactions
te = TransactionEncoder()
items = te.fit(transactions).transform(transactions)
items_df = pd.DataFrame(items, columns=te.columns_)

In [19]:
items_df.to_csv('~/ecomm-open-cdp/items_ohe_2019_oct.csv')

In [20]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483608 entries, 0 to 483607
Columns: 121 entries, accessories.bag to stationery.cartrige
dtypes: bool(121)
memory usage: 55.8 MB


In [21]:
items_df.to_parquet('~/ecomm-open-cdp/items_ohe_2019_oct.parquet')